<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/022_ParseResponse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🔍 **Why Do We Need to Parse the LLM’s Response?**

### 1️⃣ **LLMs speak in natural language. Tools need structured commands.**

LLMs generate free-form text. But tools (like file systems, APIs, or plugins) require:

```json
{
  "tool_name": "get_weather",
  "args": {
    "city": "Toronto"
  }
}
```

Without parsing, the model might say:

> “To get the weather, I’ll call the weather API with ‘Toronto’.”

… but there’s **no way to extract that into an actual API call** reliably unless it’s structured.

Parsing bridges the gap between:

> 💬 Language → ⚙️ Action

---

### 2️⃣ **Parsing adds structure = automation**

Once you define a response format like:

````markdown
```action
{ "tool_name": "...", "args": { ... } }
````

You can:

* Detect what the agent wants to do.
* Route to the correct function or API.
* Automatically execute actions without human intervention.

Without parsing, you're stuck trying to “guess” what the LLM meant.
With parsing, the LLM becomes a **reliable controller**.

---

### 3️⃣ **Parsing enables error recovery and debugging**

If the response fails to parse (bad JSON, missing fields), you can:

* Catch the error early.
* Ask the LLM to correct itself.
* Log the mistake or auto-correct it.

Without this step, the agent could silently fail or produce junk — and you’d have no clean way to fix it.

---

### 4️⃣ **Parsing lets you enforce expectations**

You're telling the LLM:

> “You must reply with a JSON block inside a \`\`\`action markdown block.”

This teaches the LLM to behave like a programmable component, not just a chat buddy.

It's the first step toward **structured, reliable, safe agent behavior**.

---

## 🎁 **Benefits of Parsing**

| Benefit         | Why it Matters                                         |
| --------------- | ------------------------------------------------------ |
| ✅ Automation    | You can auto-trigger tools based on structured output  |
| ✅ Safety        | You can catch and handle errors before execution       |
| ✅ Debuggability | Easy to log, test, and trace agent decisions           |
| ✅ Agent Control | You tell the LLM exactly how to respond and enforce it |
| ✅ Integration   | Parsed actions can plug into APIs, databases, etc.     |

---

## 🧠 TL;DR

> **Parsing is how you transform LLM output from “text” into “commands.”**
> It’s what turns your language model into a reliable **autonomous agent**.




In [1]:
%pip install -qU dotenv openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 16.4 MB/s eta 0:00:00


Let’s build a **simple agent that illustrates parsing + action execution** clearly and interactively.

---

## 🧩 **Mini Agent: Calculator Agent**

We’ll build a small agent that:

1. Takes a user prompt like: `"Divide 81 by 9"`
2. The LLM responds with a **structured tool call** (in a code block)
3. We **parse** the tool\_name and args
4. We **run the tool function** (like a real calculator!)
5. Show the result — or handle errors if the LLM formats it wrong



In [5]:
# 📚 Notebook 2: Parse + Act – Calculator Agent Demo

from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ===============================
# 1️⃣ Call the LLM
# ===============================
def generate_response(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content

# ===============================
# 2️⃣ Extract JSON from code block
# ===============================
def extract_markdown_block(text: str, tag: str = "action") -> str:
    pattern = rf"```{tag}\s*(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        raise ValueError(f"Missing markdown block with tag '{tag}'")

# ===============================
# 3️⃣ Parse the LLM response
# ===============================
def parse_action(response: str) -> dict:
    try:
        block = extract_markdown_block(response, "action")
        parsed = json.loads(block)
        if "tool_name" in parsed and "args" in parsed:
            return parsed
        else:
            return {
                "tool_name": "error",
                "args": {"message": "Response missing tool_name or args"}
            }
    except Exception as e:
        return {
            "tool_name": "error",
            "args": {"message": str(e)}
        }

# ===============================
# 4️⃣ Define calculator tools
# ===============================
def add(a, b): return a + b
def subtract(a, b): return a - b
def multiply(a, b): return a * b
def divide(a, b): return a / b if b != 0 else "Division by zero"

TOOLS = {
    "add": add,
    "subtract": subtract,
    "multiply": multiply,
    "divide": divide
}

# ===============================
# 5️⃣ Agent loop
# ===============================
system_prompt = """You are a calculator agent. Always respond with a JSON action block.
Use one of the tools: add, subtract, multiply, divide.

Respond in this format:

```action
{
  "tool_name": "add",
  "args": { "a": 5, "b": 3 }
}
```"""

conversation = [
    {"role": "system", "content": system_prompt}
]

while True:
    user_input = input("\n🧑‍💻 You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Exiting.")
        break

    conversation.append({"role": "user", "content": user_input})
    llm_reply = generate_response(conversation)
    print("\n🤖 Raw LLM Reply:\n", llm_reply)

    action = parse_action(llm_reply)
    print("\n🧠 Parsed Action:", action)

    tool_name = action.get("tool_name")
    args = action.get("args", {})

    if tool_name in TOOLS:
        result = TOOLS[tool_name](**args)
        print(f"\n⚙️ Tool Result: {result}")
    else:
        print(f"\n🚫 Invalid or missing tool: {tool_name}")



🧑‍💻 You: what is 15 minus 3?

🤖 Raw LLM Reply:
 ```action
{
  "tool_name": "subtract",
  "args": { "a": 15, "b": 3 }
}
```

🧠 Parsed Action: {'tool_name': 'subtract', 'args': {'a': 15, 'b': 3}}

⚙️ Tool Result: 12

🧑‍💻 You: exit
👋 Exiting.


## Understanding the Repsonse

Seeing the full raw `response` object from the OpenAI API will give you a **clear picture of what’s returned**, so you know how we’re getting to `response.choices[0].message.content`.

Let’s walk through it step-by-step:


## 🔍 Breaking it down

| Field                                 | What it contains                                       |
| ------------------------------------- | ------------------------------------------------------ |
| `response`                            | A `ChatCompletion` object — wraps the whole response   |
| `response.choices`                    | A list of response options (usually just 1)            |
| `response.choices[0].message`         | A `ChatMessage` with `role` and `content`              |
| `response.choices[0].message.content` | ✅ The actual text that the LLM wrote (our tool action) |

---

## 🧠 Why this matters

* The `choices` array exists because OpenAI supports **multiple completions** (e.g., top 3 ideas).
* But we usually just grab the first one: `choices[0]`.
* That’s where the LLM writes its natural language (or structured) response.




In [4]:
def generate_response(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=500
    )

    # 🔍 Print the entire response object
    print("\n🧾 Full raw response:")
    print(response)  # This is a full OpenAI object

    # 🔍 Also print just the choices list
    print("\n📦 response.choices:")
    print(response.choices)

    # 🔍 Then print the specific message object
    print("\n💬 response.choices[0].message:")
    print(response.choices[0].message)

    return response.choices[0].message.content

while True:
    user_input = input("\n🧑‍💻 You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Exiting.")
        break

    conversation.append({"role": "user", "content": user_input})
    llm_reply = generate_response(conversation)
    print("\n🤖 Raw LLM Reply:\n", llm_reply)

    action = parse_action(llm_reply)
    print("\n🧠 Parsed Action:", action)

    tool_name = action.get("tool_name")
    args = action.get("args", {})

    if tool_name in TOOLS:
        result = TOOLS[tool_name](**args)
        print(f"\n⚙️ Tool Result: {result}")
    else:
        print(f"\n🚫 Invalid or missing tool: {tool_name}")



🧑‍💻 You: What is 7 times 7?

🧾 Full raw response:
ChatCompletion(id='chatcmpl-Bu25g4hJcaIEXkOuioyP4JxGZCzKx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```action\n{\n  "tool_name": "multiply",\n  "args": { "a": 7, "b": 7 }\n}\n```', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1752694104, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=108, total_tokens=139, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

📦 response.choices:
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```action\n{\n  "tool_name": "multiply",\n  "args": { "a": 7,

Let’s refactor our calculator agent to:

✅ Use the **`parse_action()`** with `"tool_name": "error"`
✅ Let the agent **detect invalid responses**
✅ Send a follow-up message to the LLM to correct its output
✅ Handle errors safely and clearly

---

# 🔧 Step-by-step Refactor Plan

---

## ✅ **Step 1: Add a robust `parse_action()` function**

This replaces our original, simpler version:

```python
def parse_action(response: str) -> dict:
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {
                "tool_name": "error",
                "args": {"message": "Missing 'tool_name' or 'args' keys."}
            }
    except json.JSONDecodeError:
        return {
            "tool_name": "error",
            "args": {"message": "Invalid JSON. You must respond with a valid JSON tool call inside a markdown block."}
        }
    except Exception as e:
        return {
            "tool_name": "error",
            "args": {"message": str(e)}
        }
```

---

## ✅ **Step 2: Handle `tool_name == "error"` in the agent loop**

When the parser returns an error, we’ll:

* Log the error
* Feed it back to the LLM as a `"user"` message
* Try again once (for now, to keep it simple)

---

## ✅ **Step 3: Revised Agent Loop with Self-Correction**





In [6]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def generate_response(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content

def extract_markdown_block(text: str, tag: str = "action") -> str:
    pattern = rf"```{tag}\s*(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        raise ValueError(f"Missing markdown block with tag '{tag}'")

def parse_action(response: str) -> dict:
    try:
        block = extract_markdown_block(response, "action")
        parsed = json.loads(block)
        if "tool_name" in parsed and "args" in parsed:
            return parsed
        else:
            return {
                "tool_name": "error",
                "args": {"message": "Missing 'tool_name' or 'args' in response."}
            }
    except json.JSONDecodeError:
        return {
            "tool_name": "error",
            "args": {"message": "Invalid JSON. Expected JSON inside a markdown block labeled ```action```."}
        }
    except Exception as e:
        return {
            "tool_name": "error",
            "args": {"message": str(e)}
        }

# Tools
def add(a, b): return a + b
def subtract(a, b): return a - b
def multiply(a, b): return a * b
def divide(a, b): return a / b if b != 0 else "Division by zero"

TOOLS = {
    "add": add,
    "subtract": subtract,
    "multiply": multiply,
    "divide": divide
}

# Prompt
system_prompt = """You are a calculator agent. Always respond with a JSON action block.
Use one of the tools: add, subtract, multiply, divide.

Respond in this format:
```action
{
  "tool_name": "add",
  "args": { "a": 5, "b": 3 }
}
```"""

conversation = [
    {"role": "system", "content": system_prompt}
]

# 🔁 Agent loop
while True:
    user_input = input("\n🧑‍💻 You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Exiting.")
        break

    conversation.append({"role": "user", "content": user_input})
    llm_response = generate_response(conversation)
    print("\n🤖 Raw LLM Output:\n", llm_response)

    action = parse_action(llm_response)

    if action["tool_name"] == "error":
        print("\n🚫 Parse Error:", action["args"]["message"])
        # Feed the error back into the conversation for a retry
        conversation.append({"role": "assistant", "content": llm_response})
        conversation.append({
            "role": "user",
            "content": f"That was not a valid action block. Error: {action['args']['message']}. Please respond with a correct JSON tool call inside ```action```."
        })

        # Retry once
        llm_response = generate_response(conversation)
        print("\n🔁 Retry Output:\n", llm_response)
        action = parse_action(llm_response)

    tool_name = action.get("tool_name")
    args = action.get("args", {})

    if tool_name in TOOLS:
        result = TOOLS[tool_name](**args)
        print(f"\n⚙️ Tool Result: {result}")
    else:
        print(f"\n🚫 Unknown or still invalid tool: {tool_name}")



🧑‍💻 You: What is 15 times 4?

🤖 Raw LLM Output:
 ```action
{
  "tool_name": "multiply",
  "args": { "a": 15, "b": 4 }
}
```

⚙️ Tool Result: 60

🧑‍💻 You: What is 75 divide by 5?

🤖 Raw LLM Output:
 ```action
{
  "tool_name": "divide",
  "args": { "a": 75, "b": 5 }
}
```

⚙️ Tool Result: 15.0

🧑‍💻 You: exit
👋 Exiting.
